In [21]:
import requests
import json
from pprint import pprint
import pandas as pd
import datetime as dt
import numpy as np
from bs4 import BeautifulSoup as bs

In [22]:
from mykeys import austin_key, gkey
from dbfuncs import create_db_engine, drop_table, build_table, add_geodata
from pathnames import AustinIncome_url, austin_url, outcome_dataset, intake_dataset
from scrape_zipcodes import scrape_austin_income, zipcode_lookup, geo_lookup
from animal_data import query_animal_data

# Create the sqlite engine here with a function call. There will be multiple tables added to this db

In [23]:
engine = create_db_engine("sqlite://")

# Collect the income by zipcode data through a web scrape

In [24]:
income_df = scrape_austin_income(AustinIncome_url)

# Modify a few of the column names

In [25]:

chg_cols = {"Zip Code" : "ZipCode", "Avg. Income/H/hold": "HouseholdIncome", "National Rank" :"NationalRank"}
income_df.rename(inplace=True, columns = chg_cols)


# Build the db from Pandas with this table
# Right now, this is just in memory, not stored. Need to decide. We could just 
# recreate every time the page is loaded.

In [6]:
build_table(income_df,"austin_income", engine)

In [7]:
zipcodes = engine.execute('select * from austin_income').fetchall()

In [8]:
print(type(zipcodes[0]))
print(zipcodes[0])

<class 'sqlalchemy.engine.result.RowProxy'>
(0, '1.0', '78730', '30.363632, -97.850355', 'Austin, Texas', '4885', '$128,524.00', '#58')


In [9]:
print(zipcodes[0].ZipCode)

78730


# Now to get the animal data

In [10]:
outcome_df = query_animal_data(austin_url, outcome_dataset, "'2019-01-01T00:00:00.000'")
outcome_df.head()


,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,1 year,A777234,Dog,Pit Bull Mix,Blue/White,2017-11-25T00:00:00.000,2019-01-02T00:00:00.000,2019-01-02T00:00:00.000,Tyson,Partner,Transfer,Neutered Male
1,3 years,A783891,Dog,Border Terrier,Black/Tan,2015-11-08T00:00:00.000,2019-01-16T11:28:00.000,2019-01-16T11:28:00.000,Elizabeth T,Foster,Adoption,Spayed Female
2,10 months,A788493,Dog,Standard Poodle/Labrador Retriever,Cream,2018-04-03T00:00:00.000,2019-02-07T13:12:00.000,2019-02-07T13:12:00.000,NaN,Partner,Transfer,Neutered Male
3,4 years,A789594,Dog,Chihuahua Longhair Mix,Brown/Black,2015-02-24T00:00:00.000,2019-02-26T19:33:00.000,2019-02-26T19:33:00.000,Morena,NaN,Adoption,Spayed Female
4,10 years,A566568,Dog,Harrier Mix,Brown/Black,2009-02-12T00:00:00.000,2019-02-23T11:33:00.000,2019-02-23T11:33:00.000,Duck,NaN,Adoption,Neutered Male


In [11]:
build_table(outcome_df,"austin_animal_outcomes", engine)

In [12]:
outcomes = engine.execute('select * from austin_animal_outcomes').fetchall()[1:10]
pprint(outcomes)

[(1, '3 years', 'A783891', 'Dog', 'Border Terrier', 'Black/Tan', '2015-11-08T00:00:00.000', '2019-01-16T11:28:00.000', '2019-01-16T11:28:00.000', 'Elizabeth T', 'Foster', 'Adoption', 'Spayed Female'),
 (2, '10 months', 'A788493', 'Dog', 'Standard Poodle/Labrador Retriever', 'Cream', '2018-04-03T00:00:00.000', '2019-02-07T13:12:00.000', '2019-02-07T13:12:00.000', None, 'Partner', 'Transfer', 'Neutered Male'),
 (3, '4 years', 'A789594', 'Dog', 'Chihuahua Longhair Mix', 'Brown/Black', '2015-02-24T00:00:00.000', '2019-02-26T19:33:00.000', '2019-02-26T19:33:00.000', 'Morena', None, 'Adoption', 'Spayed Female'),
 (4, '10 years', 'A566568', 'Dog', 'Harrier Mix', 'Brown/Black', '2009-02-12T00:00:00.000', '2019-02-23T11:33:00.000', '2019-02-23T11:33:00.000', 'Duck', None, 'Adoption', 'Neutered Male'),
 (5, '1 year', 'A787340', 'Other', 'Bat Mix', 'Brown', '2018-01-11T00:00:00.000', '2019-01-11T14:53:00.000', '2019-01-11T14:53:00.000', None, 'Rabies Risk', 'Euthanasia', 'Unknown'),
 (6, '1 year'

In [13]:
intake_df = query_animal_data(austin_url, intake_dataset, "'2019-01-01T00:00:00.000'")
intake_df.head()
# intake_df = modify_intake_df(intake_df)

,age_upon_intake,animal_id,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,intake_type,name,sex_upon_intake
0,2 years,A786884,Dog,Beagle Mix,Tricolor,2019-01-03T16:19:00.000,2019-01-03T16:19:00.000,2501 Magin Meadow Dr in Austin (TX),Normal,Stray,*Brock,Neutered Male
1,3 days,A787254,Dog,Black Mouth Cur Mix,Black/White,2019-01-10T10:19:00.000,2019-01-10T10:19:00.000,2000 Man O War in Travis (TX),Normal,Stray,*Sprinkles,Intact Female
2,7 years,A760116,Dog,German Shepherd,Brown/Black,2019-01-07T12:07:00.000,2019-01-07T12:07:00.000,Austin (TX),Normal,Owner Surrender,Jasmine,Spayed Female
3,3 years,A789406,Cat,Domestic Shorthair Mix,Brown Tabby,2019-02-21T14:55:00.000,2019-02-21T14:55:00.000,Austin (TX),Normal,Public Assist,Liliana,Intact Female
4,4 years,A787054,Dog,Labrador Retriever Mix,Black/White,2019-01-24T15:17:00.000,2019-01-24T15:17:00.000,Austin (TX),Normal,Owner Surrender,*Rexx,Neutered Male


In [14]:
build_table(intake_df,"austin_animal_intake", engine)

In [15]:
intake = engine.execute('select * from austin_animal_intake').fetchall()[1:10]
pprint(intake)

[(1, '3 days', 'A787254', 'Dog', 'Black Mouth Cur Mix', 'Black/White', '2019-01-10T10:19:00.000', '2019-01-10T10:19:00.000', '2000 Man O War in Travis (TX)', 'Normal', 'Stray', '*Sprinkles', 'Intact Female'),
 (2, '7 years', 'A760116', 'Dog', 'German Shepherd', 'Brown/Black', '2019-01-07T12:07:00.000', '2019-01-07T12:07:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', 'Jasmine', 'Spayed Female'),
 (3, '3 years', 'A789406', 'Cat', 'Domestic Shorthair Mix', 'Brown Tabby', '2019-02-21T14:55:00.000', '2019-02-21T14:55:00.000', 'Austin (TX)', 'Normal', 'Public Assist', 'Liliana', 'Intact Female'),
 (4, '4 years', 'A787054', 'Dog', 'Labrador Retriever Mix', 'Black/White', '2019-01-24T15:17:00.000', '2019-01-24T15:17:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', '*Rexx', 'Neutered Male'),
 (5, '10 years', 'A788104', 'Dog', 'Beagle Mix', 'Tricolor/Brown', '2019-01-26T14:04:00.000', '2019-01-26T14:04:00.000', '14200 Ranch Road 620 in Austin (TX)', 'Normal', 'Stray', 'Beaux', 'Neutere

# Code to convert address to lat/long and zip code, and add these three new columns to the intake database

In [16]:
addresses = engine.execute('select animal_id, found_location from austin_animal_intake').fetchall()[1:200]

In [17]:
print(addresses[0].animal_id)

A787254


In [18]:
add_geodata(addresses, engine)

In [19]:
test_df = pd.read_sql_query("SELECT * FROM austin_animal_intake WHERE zipcode == 'None'", con=engine)

In [20]:
test_df

,index,age_upon_intake,animal_id,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,intake_type,name,sex_upon_intake,lat,long,zipcode


In [40]:
test_df.to_csv("Resources/test_intake_data.csv")

In [41]:
test2_df = pd.read_sql_query("SELECT * from austin_animal_outcomes", con=engine)

In [42]:
test2_df.to_csv("Resources/test_outcomes_data.csv")

In [70]:
select_str = "select i.*, \
              o.age_upon_outcome, o.date_of_birth, o.datetime as outcome_datetime, \
              o.outcome_subtype, o.outcome_type, o.sex_upon_outcome, m.HouseholdIncome \
              from austin_animal_intake as i left join austin_animal_outcomes as o on i.animal_id = o.animal_id \
              inner join austin_income as m where i.zipcode = m.ZipCode"
              
select_df = pd.read_sql_query(select_str, con=engine)

In [71]:
select_df.head()

,index,age_upon_intake,animal_id,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,...,lat,long,zipcode,age_upon_outcome,date_of_birth,outcome_datetime,outcome_subtype,outcome_type,sex_upon_outcome,HouseholdIncome
0,1,3 days,A787254,Dog,Black Mouth Cur Mix,Black/White,2019-01-10T10:19:00.000,2019-01-10T10:19:00.000,2000 Man O War in Travis (TX),Normal,...,30.3475374,-97.7216636,78757,2 months,2019-01-07T00:00:00.000,2019-03-19T11:00:00.000,Foster,Adoption,Spayed Female,"$45,090.00"
1,2,7 years,A760116,Dog,German Shepherd,Brown/Black,2019-01-07T12:07:00.000,2019-01-07T12:07:00.000,Austin (TX),Normal,...,30.267153,-97.7430608,78702,7 years,2011-05-11T00:00:00.000,2019-01-10T19:20:00.000,None,Adoption,Spayed Female,"$23,348.00"
2,2,7 years,A760116,Dog,German Shepherd,Brown/Black,2019-01-07T12:07:00.000,2019-01-07T12:07:00.000,Austin (TX),Normal,...,30.267153,-97.7430608,78702,7 years,2011-05-11T00:00:00.000,2019-02-09T13:14:00.000,None,Adoption,Spayed Female,"$23,348.00"
3,3,3 years,A789406,Cat,Domestic Shorthair Mix,Brown Tabby,2019-02-21T14:55:00.000,2019-02-21T14:55:00.000,Austin (TX),Normal,...,30.267153,-97.7430608,78702,3 years,2015-08-21T00:00:00.000,2019-03-04T15:58:00.000,None,Return to Owner,Spayed Female,"$23,348.00"
4,4,4 years,A787054,Dog,Labrador Retriever Mix,Black/White,2019-01-24T15:17:00.000,2019-01-24T15:17:00.000,Austin (TX),Normal,...,30.267153,-97.7430608,78702,4 years,2015-01-07T00:00:00.000,2019-01-20T15:02:00.000,None,Adoption,Neutered Male,"$23,348.00"


In [73]:
select_df.to_csv("Resources/fulldata.csv")